In [19]:
import sys
import platform
import locale
import dbDictUpload as db
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection
import certifi
from elasticsearch import helpers
from keysLegislation import elastic_keys
import insta_scraper as insta
from insta_listkeys import keys
import insta_dictionaries as idict
from InstagramAPI import InstagramAPI
from tags import tag_posts
import insta_s3 as is3

import logging
from datetime import datetime

logger = logging.getLogger(__name__)

In [20]:
def location_geopoint(d):
    if "location_lat" in d.keys():
        logger.info("Location: {}".format(d["url"]))
        d["pin"] = {
            "location" : {
                "lat" : d["location_lat"],
                "lon" : d["location_lng"]
                }
            }

    return d

In [17]:
def main():
    logger.info('Init dicts and database connections...')
    path = ''
    es = Elasticsearch(elastic_keys['endpoint'],
                          http_auth=(elastic_keys['usr'], elastic_keys['pwd']),
                          timeout=30, max_retries=10,retry_on_timeout=True)
    
    API = InstagramAPI(keys[2]['user_name'],keys[2]['pwd'])
    API.login()
    for topic,tags in tag_posts.items():
        for tag in tags:
            try:
                logger.info('Tag: {}. Getting media...'.format(tag))
                posts_story = insta.get_tag_feed(tag, API,type_ = 'story')
                logger.info('{} posts'.format(len(posts_story)))
                count_uploded_to_elastic, i = 0, 1 
                for media in posts_story:
                    logger.info('{}/{}'.format(i,len(posts_story)))
                    i += 1
                    logger.info('Building entities...')
                    d = idict.media_story_dict(media)
                    d = location_geopoint(d)
                    d['topic'] = topic
                    d['source'] = 'story'
                    #save story in s3
                    logger.info('Saving in s3')
                    bucket = "insta-stories"
                    s3dir = 'stories-photos'
                    print('='*100)
                    #print(d['url'])
                    print(s3dir,d['insta_id2'],path)
                    print('='*100)
                    #is3.save_story(d['url'],s3dir,d['insta_id2'],path+"/insta_scripts")
                    s3link = "https://s3-us-west-2.amazonaws.com/{}/{}/{}.{}".format(bucket,s3dir,d['insta_id2'],"png")
                    d['s3_photo'] = s3link
                    if 'video_url' in d.keys():
                        s3dir = 'stories-videos'
                        #is3.save_story(d['video_url'],s3dir,d['insta_id2'],path+"/insta_scripts")
                        s3link = "https://s3-us-west-2.amazonaws.com/{}/{}/{}.{}".format(bucket,s3dir,d['insta_id2'],"mp4")
                        d['s3_video'] = s3link
                    #upload to elastic
                    logger.info('Uploading...')
                    d['timestamp'] = datetime.now()
                    print('tattoos','instagram-locations',d,str(d['insta_id2']))
                    #uploaded_elastic = db.single_upload_elastic(es,'tattoos','instagram-locations',d,id_=str(d['insta_id2']))
                    #if uploaded_elastic:
                    #    count_uploded_to_elastic += 1
                #logger.info('{}/{} posts uploaded to elastic for {}'.format(count_uploded_to_elastic,len(posts_story),tag))
                print("Ah?")
            except:
                #print("Ah?")
                logging.exception('CRITICAL ERROR uploading posts for: {}'.format(tag,str(sys.exc_info()[0])))
    API.logout()
    logger.info("Finished!!")

In [18]:
if __name__ == "__main__":
    format_t = "%(asctime)s [%(levelname)s] %(message)s"
    logging.basicConfig(format=format_t,level=logging.INFO)#filename='congressesMain.log')
    main()

2019-02-11 16:17:03,570 [INFO] Init dicts and database connections...
2019-02-11 16:17:04,132 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-11 16:17:04,352 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-11 16:17:05,444 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthe

Request return 404 error!


2019-02-11 16:17:07,286 [INFO] Tag: tattoos. Getting media...
2019-02-11 16:17:07,374 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Login success!



2019-02-11 16:17:09,619 [INFO] 16 posts
2019-02-11 16:17:09,621 [INFO] 1/16
2019-02-11 16:17:09,622 [INFO] Building entities...
2019-02-11 16:17:09,623 [INFO] Saving in s3
2019-02-11 16:17:09,625 [INFO] Uploading...
2019-02-11 16:17:09,627 [INFO] 2/16
2019-02-11 16:17:09,628 [INFO] Building entities...
2019-02-11 16:17:09,630 [INFO] Saving in s3
2019-02-11 16:17:09,631 [INFO] Uploading...
2019-02-11 16:17:09,632 [INFO] 3/16
2019-02-11 16:17:09,634 [INFO] Building entities...
2019-02-11 16:17:09,635 [INFO] Location: https://instagram.fmex6-1.fna.fbcdn.net/vp/2316c11f31dd326078c03de30d8854ec/5C6464F2/t51.12442-15/e15/p640x640/50722787_633536067104130_3217353017857059695_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net
2019-02-11 16:17:09,636 [INFO] Saving in s3
2019-02-11 16:17:09,640 [INFO] Uploading...
2019-02-11 16:17:09,641 [INFO] 4/16
2019-02-11 16:17:09,642 [INFO] Building entities...
2019-02-11 16:17:09,643 [INFO] Saving in s3
2019-02-11 16:17:09,644 [INFO] Uploading...
2019-02-11 16:

...

2019-02-11 16:17:14,745 [INFO] 20 posts
2019-02-11 16:17:14,748 [INFO] 1/20
2019-02-11 16:17:14,749 [INFO] Building entities...
2019-02-11 16:17:14,750 [INFO] Location: https://instagram.fmex6-1.fna.fbcdn.net/vp/a2c1757dc8a061509a0af56a35d8cb9c/5C641A54/t51.12442-15/e15/p640x640/50771819_1661370967296695_3153283087248330667_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net
2019-02-11 16:17:14,752 [INFO] Saving in s3
2019-02-11 16:17:14,753 [INFO] Uploading...
2019-02-11 16:17:14,755 [INFO] 2/20
2019-02-11 16:17:14,756 [INFO] Building entities...
2019-02-11 16:17:14,757 [INFO] Saving in s3
2019-02-11 16:17:14,759 [INFO] Uploading...
2019-02-11 16:17:14,761 [INFO] 3/20
2019-02-11 16:17:14,762 [INFO] Building entities...
2019-02-11 16:17:14,763 [INFO] Saving in s3
2019-02-11 16:17:14,764 [INFO] Uploading...
2019-02-11 16:17:14,765 [INFO] 4/20
2019-02-11 16:17:14,766 [INFO] Building entities...
2019-02-11 16:17:14,767 [INFO] Saving in s3
2019-02-11 16:17:14,768 [INFO] Uploading...
2019-02-11 16

stories-photos 1976953778854786585_2075246094 
tattoos instagram-locations {'supports_reel_reactions': True, 'user_profile_pic_id': '1961157260991691943_2075246094', 'number_of_qualities': 3, 'is_reel_media': True, 'date': datetime.datetime(2019, 2, 11, 7, 21, 9), 'user_is_private': False, 'user_is_verified': False, 'user_id': 2075246094, 'source': 'story', 'pin': {'location': {'lat': 7.12972, 'lon': -73.1258}}, 'insta_source': 'tag', 'video_codec': 'avc1.4D401F', 'text': None, 'media_type': 2, 'story_hashtags': [{'y': 0.436447948633001, 'z': 1, 'is_hidden': 0, 'is_pinned': 0, 'hashtag': {'id': 17843913331008006, 'name': 'tatuajes'}, 'x': 0.7940646403509131, 'width': 0.133592444977496, 'height': 0.030377430213039004, 'rotation': 0.0}], 's3_photo': 'https://s3-us-west-2.amazonaws.com/insta-stories/stories-photos/1976953778854786585_2075246094.png', 'insta_id2': '1976953778854786585_2075246094', 'video_duration': 5.0, 'user_profile_pic_url': 'https://instagram.fmex6-1.fna.fbcdn.net/vp/ca

2019-02-11 16:17:14,976 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-11 16:17:17,231 [INFO] 17 posts
2019-02-11 16:17:17,232 [INFO] 1/17
2019-02-11 16:17:17,232 [INFO] Building entities...
2019-02-11 16:17:17,233 [INFO] Location: https://instagram.fmex6-1.fna.fbcdn.net/vp/c5f0633c5accc32423c38aea4b8d1759/5C6421A5/t51.12442-15/e15/p640x640/50786115_2300832986797357_6639859273643758100_n.jpg?_nc_ht=instagram.fmex6-1.fna.fbcdn.net
2019-02-11 16:17:17,234 [INFO] Saving in s3
2019-02-11 16:17:17,235 [INFO] Uploading...
2019-02-11 16:17:17,237 [INFO] 2/17
2019-02-11 16:17:17,237 [INFO] Building entities...
2019-02-11 16:17:17,238 [INFO] Saving in s3
2019-02-11 16:17:17,239 [INFO] Uploading...
2019-02-11 16:17:17,241 [INFO

stories-photos 1976848799855400648_2119203114 
tattoos instagram-locations {'supports_reel_reactions': True, 'user_profile_pic_id': '1910265224709516624_2119203114', 'number_of_qualities': 4, 'is_reel_media': True, 'date': datetime.datetime(2019, 2, 11, 3, 52, 41), 'user_is_private': False, 'user_is_verified': False, 'user_id': 2119203114, 'source': 'story', 'pin': {'location': {'lat': 41.3888, 'lon': 2.1681}}, 'insta_source': 'tag', 'video_codec': 'avc1.4D401F', 'text': None, 'media_type': 2, 'story_hashtags': [{'y': 0.7209881332111561, 'z': 2, 'is_hidden': 0, 'is_pinned': 0, 'hashtag': {'id': 17841563908116504, 'name': 'tatuaje'}, 'x': 0.8395707532955121, 'width': 0.24971211684379102, 'height': 0.06309302974445401, 'rotation': 0.026881035591924}], 's3_photo': 'https://s3-us-west-2.amazonaws.com/insta-stories/stories-photos/1976848799855400648_2119203114.png', 'insta_id2': '1976848799855400648_2119203114', 'video_duration': 5.0, 'user_profile_pic_url': 'https://instagram.fmex6-1.fna.f

2019-02-11 16:17:17,443 [WARNING] /home/ubuntu/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

2019-02-11 16:17:17,758 [INFO] Finished!!
